#Музыка для фильма

##Доп. настройки

In [ ]:
#@title Выберете жанр (тот же, что и в тексте):

drama=True #@param {type:"boolean"}
sci_fi = False #@param {type:"boolean"}
horror = False #@param {type:"boolean"}
thriller = False #@param {type:"boolean"}
action = False #@param {type:"boolean"}
superhero = False #@param {type:"boolean"}

In [ ]:
#@title Подключите диск при необходимости:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import soundfile as sf
path = f'/content/voice.wav' #путь до файла с голосовым озвучиванием
file = sf.SoundFile(path)
duration = len(file)/file.samplerate #узнаём длинну этого файла

##Запускать только 1 из моделей

## multitrack music vae
### **drama, sci-fi, horror, thriller**

### Environment Setup

In [ ]:
#@title Setup Environment

print('Copying checkpoints and modified SGM SoundFont (https://sites.google.com/site/soundfonts4u) from GCS.')
print('This will take a few minutes...')
!gsutil -q -m cp gs://download.magenta.tensorflow.org/models/music_vae/multitrack/* /content/
!gsutil -q -m cp gs://download.magenta.tensorflow.org/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 /content/

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -qU magenta pyfluidsynth pretty_midi

print('Importing libraries...')

import numpy as np
import os
import tensorflow.compat.v1 as tf

from google.colab import files

import magenta.music as mm
from magenta.music.sequences_lib import concatenate_sequences
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

tf.disable_v2_behavior()

In [ ]:
#@title Definitions

BATCH_SIZE = 4
Z_SIZE = 512
TOTAL_STEPS = 512
BAR_SECONDS = 2.0
CHORD_DEPTH = 49

SAMPLE_RATE = 44100
SF2_PATH = '/content/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2'

# Play sequence using SoundFont.
def play(note_sequences):
  if not isinstance(note_sequences, list):
    note_sequences = [note_sequences]
  for ns in note_sequences:
    mm.play_sequence(ns, synth=mm.fluidsynth, sf2_path=SF2_PATH)
  
# Spherical linear interpolation.
def slerp(p0, p1, t):
  """Spherical linear interpolation."""
  omega = np.arccos(np.dot(np.squeeze(p0/np.linalg.norm(p0)), np.squeeze(p1/np.linalg.norm(p1))))
  so = np.sin(omega)
  return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

# Download sequence.
def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  #files.download(filename)

# Chord encoding tensor.
def chord_encoding(chord):
  index = mm.TriadChordOneHotEncoding().encode_event(chord)
  c = np.zeros([TOTAL_STEPS, CHORD_DEPTH])
  c[0,0] = 1.0
  c[1:,index] = 1.0
  return c

# Trim sequences to exactly one bar.
def trim_sequences(seqs, num_seconds=BAR_SECONDS):
  for i in range(len(seqs)):
    seqs[i] = mm.extract_subsequence(seqs[i], 0.0, num_seconds)
    seqs[i].total_time = num_seconds

# Consolidate instrument numbers by MIDI program.
def fix_instruments_for_concatenation(note_sequences):
  instruments = {}
  for i in range(len(note_sequences)):
    for note in note_sequences[i].notes:
      if not note.is_drum:
        if note.program not in instruments:
          if len(instruments) >= 8:
            instruments[note.program] = len(instruments) + 2
          else:
            instruments[note.program] = len(instruments) + 1
        note.instrument = instruments[note.program]
      else:
        note.instrument = 9

In [ ]:
import wave

### Chord-Conditioned Model

In [ ]:
#@title Выбераем аккорды

if drama:
 chord_1 = 'Cm'
 chord_2 = 'E'
 chord_3 = 'B'
 chord_4 = 'Cm'

if sci_fi:
 chord_1 = 'Cm'
 chord_2 = 'F'
 chord_3 = 'B'
 chord_4 = 'Am'
 
if horror or thriller:
 chord_1 = 'C'
 chord_2 = 'C'
 chord_3 = 'B'
 chord_4 = 'Dm'

In [ ]:
#@title Загружаем чекпоинт

config = configs.CONFIG_MAP['hier-multiperf_vel_1bar_med_chords']
model = TrainedModel(
    config, batch_size=BATCH_SIZE,
    checkpoint_dir_or_path='/content/model_chords_fb64.ckpt')

In [ ]:
#@title Генерация музыки
filename = 'music' #@param {type:'string'}

for i in range(int(duration//120)+1): #для улучшния качества генерится несколько мелодий, каждая по 2 мин
  chords = [chord_1, chord_2, chord_3, chord_4]

  num_bars = 64
  temperature = 0.2 

  z1 = np.random.normal(size=[Z_SIZE])
  z2 = np.random.normal(size=[Z_SIZE])
  z = np.array([slerp(z1, z2, t)
              for t in np.linspace(0, 1, num_bars)])

  seqs = [
    model.decode(length=TOTAL_STEPS, z=z[i:i+1, :], temperature=temperature,
                 c_input=chord_encoding(chords[i % 4]))[0]
    for i in range(num_bars)
  ]

  trim_sequences(seqs)
  fix_instruments_for_concatenation(seqs)
  prog_interp_ns = concatenate_sequences(seqs)

  download(prog_interp_ns, filename+str(i)+ '.mid')

  play(prog_interp_ns)
  mm.plot_sequence(prog_interp_ns)

In [ ]:
#@title mid2wav

!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

import os
import time

In [ ]:
#@title Обьединение в wav файл

outfile = 'music' #@param {type:'string'}
outfile+= '.wav'

data1 = []
i=0
while True:
  try:
    os.popen('fluidsynth -ni font.sf2 /content/'+filename + str(i)+'.mid -F '+filename + str(i)+'.wav -r 44100')#mid2wav
    time.sleep(5) #ждём когда новый файл появится в content
    w = wave.open(f'/content/{filename}{i}.wav', 'rb')
    print(i)
    data1.append([w.getparams(), w.readframes(w.getnframes())])
    w.close()
    i+=1
  except FileNotFoundError:
    break  

output = wave.open(outfile, 'wb')
output.setparams(data1[0][0])
for j in range(len(data1)):
  output.writeframes(data1[j][1]) #обьединение
output.close

In [ ]:
import IPython

IPython.display.Audio(f'/content/{outfile}')

##music vae
###**action, superhero**

###env setup

In [ ]:
#@title Setup Environment

import glob

BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

# Hack to allow python to pick up the newly-installed fluidsynth lib.
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)

  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  #files.download(filename)

print('Done')

### 16-bar Melody Models

In [ ]:
#@title Загрузка модели
mel_16bar_models = {}
hierdec_mel_16bar_config = configs.CONFIG_MAP['hierdec-mel_16bar']
mel_16bar_models['hierdec_mel_16bar'] = TrainedModel(hierdec_mel_16bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/mel_16bar_hierdec.ckpt')

flat_mel_16bar_config = configs.CONFIG_MAP['flat-mel_16bar']
mel_16bar_models['baseline_flat_mel_16bar'] = TrainedModel(flat_mel_16bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/mel_16bar_flat.ckpt')

In [ ]:
#@title Генерация
mel_sample_model = "hierdec_mel_16bar" 
temperature = 0.5

for i in range(int(duration//30)+1):
  mel_16_samples = mel_16bar_models[mel_sample_model].sample(n=1, length=50, temperature=temperature)
  for ns in mel_16_samples:
    download(ns, f'music{i}.mid')
#for ns in mel_16_samples:
#  play(ns)
#раскоментировать если хотите прослушать mid фрагменты

In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2
import wave

In [ ]:
#@title mid2wav
for j in range(i+1):
  try:
    os.popen('fluidsynth -ni font.sf2 /content/music' + str(j)+'.mid -F music'+ str(j)+'.wav -r 44100')#mid2wav
  except FileNotFoundError:
    break

In [ ]:
#@title Сборка в 1 файл
outfile = 'music' #@param {type:'string'}
outfile+= '.wav'

data1 = []
for j in range(i+1):
    w = wave.open(f'/content/music{j}.wav', 'rb')
    data1.append([w.getparams(), w.readframes(w.getnframes())])
    w.close()

In [ ]:
output = wave.open(outfile, 'wb')
output.setparams(data1[0][0])
for j in range(len(data1)):
  output.writeframes(data1[j][1])
output.close

In [ ]:
import IPython

IPython.display.Audio(f'/content/{outfile}')

# Пожалуйста сохраните получившийся музыкальный файл, он нам пригодится в "Фильм_часть3" 